In [1]:
%load_ext autoreload
%autoreload 2

import sys, os
import pandas as pd
import numpy as np
from tqdm import tqdm
import wandb

sys.path.append(os.path.join(os.getcwd(), '../symlie'))
from misc.utils import NumpyUtils, Results

In [2]:
# results_df_old = pd.read_pickle('../logs/store/results_df.pkl')
results_df_old = pd.DataFrame(columns = ['run_id'])
results_df_old

,run_id


In [3]:
log_dir = '../logs'
store_dir = os.path.join(log_dir, 'store')
wandb_dir = os.path.join(log_dir, 'wandb')

run_dirs =  np.array([[run_dir.split('-')[-1], run_dir] for run_dir in os.listdir(wandb_dir) if run_dir.startswith('run')])
if not len(run_dirs) == 0:
    assert len(run_dirs[:, 0]) == len(set(run_dirs[:, 0]))
    run_dirs = {run_id: run_dir for run_id, run_dir in run_dirs}
    run_dirs_df = pd.DataFrame(run_dirs.items(), columns = ['run_id', 'run_dir'])
else:
    run_dirs_df = pd.DataFrame(columns = ['run_id', 'run_dir'])
run_dirs_df

,run_id,run_dir
0,unpw27ml,run-20240125_154053-unpw27ml
1,yr3rrll1,run-20240125_150132-yr3rrll1
2,m6kv6per,run-20240125_145450-m6kv6per
3,h5g6wqls,run-20240125_153203-h5g6wqls
4,8ueaarbk,run-20240125_145450-8ueaarbk
...,...,...
149,s0ixufs2,run-20240125_150132-s0ixufs2
150,w02shlw6,run-20240125_153431-w02shlw6
151,e60auxdz,run-20240125_135751-e60auxdz
152,q29n51sr,run-20240125_153430-q29n51sr


In [4]:
exclude_ids = []

In [5]:
# for run in runs:
#     run_id = run.id

#     run_dir = run_dirs[run_id]

#     file_src = os.path.join(wandb_dir, run_dir, 'files', 'P.npy')
#     print(file_src)
#     file_dest = os.path.join(store_dir, 'P', f'{run_id}.npy')
#     print(file_dest)
#     shutil.move(file_src, file_dest)

In [6]:
tags = 'v5'

api = wandb.Api()
filters = {'tags': tags} if tags else {}
filters['state'] = 'finished'
runs = api.runs('eliasdubbeldam/symlie', filters=filters)
print(len(runs))
for i, run in enumerate(runs):
    print(run.name, run.id)
    # if run.id in exclude_ids: run.delete()

40
worldly-snowflake-183 2m7eu5tn
classic-glade-184 ugylvzz6
sage-river-182 unpw27ml
azure-grass-181 w4yjcu25
blooming-star-180 q86f7s5e
electric-cosmos-179 23tbcas1
lyric-blaze-178 vw4gomv4
snowy-haze-177 r6ilnqy7
tough-dream-175 wumlcc9g
fine-frog-176 t0m1gmbl
northern-planet-173 vcesgz9i
revived-blaze-174 iy9dn0r8
usual-field-172 qgqnd8cc
leafy-night-170 ysqijcn6
glorious-night-170 a7hwxl4q
copper-totem-165 pvz0o5gk
eager-river-168 axdywa74
pious-breeze-166 aqlbpqyk
glorious-glitter-169 7amet5uu
silver-sponge-165 58ksq3it
glad-darkness-163 qgj6m8fp
zesty-water-164 xvs142jj
hopeful-planet-161 jm10pkgk
rich-vortex-161 1jgutjyw
grateful-tree-160 u0l44x06
major-lion-149 e5alcris
fancy-armadillo-158 j0eaex2q
trim-shape-148 fmoh4b7x
cerulean-salad-154 dm3gzj4p
rosy-fog-156 d27txfs7
happy-pond-156 9oncfms3
stoic-armadillo-150 79km8poo
helpful-water-153 w02shlw6
prime-aardvark-151 wmh14rq2
divine-wind-151 12emq1ti
young-wind-158 xs7l5aot
cool-field-154 056mlps1
eternal-pine-146 im401q4l
fra

In [7]:
listdir_dirs = lambda path: [entry for entry in os.listdir(path) if os.path.isdir(os.path.join(path, entry))]
store_dir_dirs = listdir_dirs(store_dir)
store_dir_dirs

['P']

In [9]:
config_list = []

pbar = tqdm(runs)
for run in pbar:
    pbar.set_description(f'Retreiving wandb {str(id)}')
    
    id = run.id

    # Exceptions
    if run.state != 'finished':
        continue
    if id in results_df_old['run_id'].values:
        continue
    if id in exclude_ids:
        continue


    # Retreive config from wandb run, add all info to config
    config = run.config

    # Add run id and name to config
    config['run_id'] = id
    config['run_name'] = run.name

    # Retreive metrics from wandb run
    history = run.history()
    metrics = ['test_loss'] # 'train_loss', 'val_loss'
    for metric in metrics:
        config[metric] = np.array(history[metric].dropna())

    # Retreive arrays from wandb files
    # npu = NumpyUtils(dir = os.path.join(wandb_dir, run_dirs[id], 'files'))
    # run_arrays = npu.load_all()
    # for filename, array in run_arrays.items():
    #     config[filename] = array
    
    # for store_dir_dir in store_dir_dirs:
    #     array = np.load(os.path.join(store_dir, store_dir_dir, f'{id}.npy'))    
    #     config[store_dir_dir] = array


    config_list.append(config)

results_df = pd.DataFrame(config_list)
results_df

Retreiving wandb q29n51sr: 100%|██████████| 40/40 [00:15<00:00,  2.61it/s]


,lr,net,bias,name,seed,tags,test,n_val,train,y_low,...,num_workers,space_length,generate_data,model_summary,args_processed,do_return_model,transform_kwargs,persistent_workers,run_name,test_loss
0,0.0010,TrainP,False,None,5,[v5],True,100,True,1,...,18,7,False,False,True,False,"{'augment': 'transform_flower', 'eps_mult': [0...",True,worldly-snowflake-183,[0.0004728641652036458]
1,0.0001,TrainP,False,None,5,[v5],True,100,True,1,...,18,7,False,False,True,False,"{'augment': 'transform_flower', 'eps_mult': [0...",True,classic-glade-184,[0.009488677605986595]
2,0.1000,TrainP,False,None,5,[v5],True,100,True,1,...,18,7,False,False,True,False,"{'augment': 'transform_flower', 'eps_mult': [0...",True,sage-river-182,[0.0035890007857233286]
3,0.0100,TrainP,False,None,5,[v5],True,100,True,1,...,18,7,False,False,True,False,"{'augment': 'transform_flower', 'eps_mult': [0...",True,azure-grass-181,[0.0031849860679358244]
4,0.0001,TrainP,False,None,5,[v5],True,100,True,1,...,18,7,False,False,True,False,"{'augment': 'transform_flower', 'eps_mult': [0...",True,blooming-star-180,[0.02364257164299488]
5,0.0010,TrainP,False,None,5,[v5],True,100,True,1,...,18,7,False,False,True,False,"{'augment': 'transform_flower', 'eps_mult': [0...",True,electric-cosmos-179,[0.011470845900475979]
6,0.1000,TrainP,False,None,5,[v5],True,100,True,1,...,18,7,False,False,True,False,"{'augment': 'transform_flower', 'eps_mult': [0...",True,lyric-blaze-178,[0.10497400164604187]
7,0.0100,TrainP,False,None,5,[v5],True,100,True,1,...,18,7,False,False,True,False,"{'augment': 'transform_flower', 'eps_mult': [0...",True,snowy-haze-177,[0.023234985768795013]
8,0.0010,TrainP,False,None,4,[v5],True,100,True,1,...,18,7,False,False,True,False,"{'augment': 'transform_flower', 'eps_mult': [0...",True,tough-dream-175,[0.0005002270918339491]
9,0.0001,TrainP,False,None,4,[v5],True,100,True,1,...,18,7,False,False,True,False,"{'augment': 'transform_flower', 'eps_mult': [0...",True,fine-frog-176,[0.008932985365390778]


In [46]:
results_df_new = pd.concat([results_df_old, results_df])
results_df_new.to_pickle(os.path.join(store_dir, 'results_df.pkl'))